## Here are the links to the publicly available datasets that I used:

Hate Speech English Dataset: https://archive.ics.uci.edu/ml/machine-learning-databases/20newsgroups-mld/20_newsgroups.tar.gz

Reddit Hate Speech Dataset: https://github.com/jing-qian/A-Benchmark-Dataset-for-Learning-to-Intervene-in-Online-Hate-Speech



In [ ]:
# Importing the libraries:
import pandas as pd
import os
import numpy

In [ ]:
# Reading the data:
reddit = pd.read_csv("/content/drive/My Drive/Glue Labs/Data-Comments/reddit.csv")
reddit.head()

,id,text,hate_speech_idx,response
0,1. e8q18lf\n2. \te8q9w5s\n3. \t\te8qbobk\n4. \...,1. A subsection of retarded Hungarians? Ohh bo...,[1],"[""I don't see a reason why it's okay to insult..."
1,1. e9c6naz\n2. \te9d03a5\n3. \t\te9d8e4d\n,"1. > ""y'all hear sumn?"" by all means I live i...",[3],['Persons with disabilities is the accepted te...
2,1. e84rl2i\n2. \te84w60l\n3. \t\te8544rn\n4. \...,1. wouldn't the defenders or whatever they are...,NaN,NaN
3,1. e7kq72n\n2. \te7m24ar\n,1. Because the Japanese aren't retarded and kn...,[1],"[""It's not right for anyone of any gender to b..."
4,1. e7hdgoh\n2. \te7iyj6a\n3. \t\te7j6iho\n4. \...,1. That might be true if we didn't have an exa...,"[2, 3]","[""You shouldn't be bringing up sensitive topic..."


In [ ]:
# Dropping useless columns:
reddit.drop(['id', 'response'], axis = 1, inplace = True)

Lets try to define a function that preprocesses a paragraph we pass through it:

In [ ]:
def preprocesstext(text):
  u = text.replace(str("[deleted]"), "")
  v = u.replace(str("[removed]"), "")
  w = v.replace(str("["), "")
  x = w.replace(str("]"), "")
  y = x.replace(str("*"), "")
  z = y.replace(str("/r/"), "")
  a = z.replace(str("/r/"), "")
  b = a.replace(str("/"), "")
  c = b.replace(str("\t"), "")
  d = c.replace(str("\n"), "")
  e = d.replace(str(1) + ".", "")
  f = e.replace(str(2) + ".", "")
  g = f.replace(str(3) + ".", "")
  h = g.replace(str(4) + ".", "")
  i = h.replace(str(5) + ".", "")
  j = i.replace(str(6) + ".", "")
  k = j.replace(str(">"), "")
  l = k.replace("    ", " ")
  m = l.replace("   ", " ")
  n = m.replace("  ", " ")
  PreprocessedText = n
  return PreprocessedText

  # Not yet dealt with symbols

In [ ]:
reddit

,text,hate_speech_idx
0,1. A subsection of retarded Hungarians? Ohh bo...,[1]
1,"1. > ""y'all hear sumn?"" by all means I live i...",[3]
2,1. wouldn't the defenders or whatever they are...,NaN
3,1. Because the Japanese aren't retarded and kn...,[1]
4,1. That might be true if we didn't have an exa...,"[2, 3]"
...,...,...
5015,1. Who does she claim as *her people*?\n2. \tI...,[8]
5016,1. Here's what's going to happen: Broward is ...,NaN
5017,1. Oh boy...so here comes a long venting post ...,"[1, 4]"
5018,"1. OP, stop being a faggot and post videos nex...",[1]


In [ ]:
# Lets take a look at this paragraph. 
reddit.text[5009]

'1. What is it with girls in "Pink" shirts always being the cunt in the situation?\n'

In [ ]:
# Now lets preprocess this:
preprocesstext(reddit.text[5009])

' What is it with girls in "Pink" shirts always being the cunt in the situation?'

Now, its time to preprocess the csv as a whole.


In [ ]:
# Checking the dimensions of our csv:
reddit.shape

(5020, 2)

In [ ]:
for n in range(5020):
  paragraph = reddit.text.iloc[n]
  ProcessedParagraph = preprocesstext(paragraph)
  reddit.text.iloc[n] = ProcessedParagraph

In [ ]:
reddit.columns = ["text", "hatescore"]

In [ ]:
reddit[reddit['hatescore'].isnull()]

,text,hatescore
2,wouldn't the defenders or whatever they are a...,NaN
10,"Hate seeing inexperienced cops work, but man ...",NaN
17,"Grammatical errors, overt racism, child prost...",NaN
19,"“These are damaged, needy, lonely women,” say...",NaN
26,"Sorry, I'm not following many news on America...",NaN
...,...,...
4987,"Just stride in confidently with a ""sup fucker...",NaN
4989,how do these cunts keep a straight face while...,NaN
4995,As a father of a daughter who has been bullie...,NaN
5005,I said Gueten Tag to a German guy. It didn't ...,NaN


We have NaN values which represent the clean tweets in there so lets replace them with a string [0]. We won't be using an integer since using a string which is similar to other values makes the preprocessing easier.

In [ ]:
NullIndex = numpy.array((reddit[reddit['hatescore'].isnull()]).index)
NullIndex = NullIndex.tolist()

In [ ]:
for i in NullIndex:
  reddit.hatescore.iloc[i] = str([0])

Defining a getscore function which gets the hatescore:

In [ ]:
def getscore(array):
  
  b = array.replace("[", "").replace("]", "").split(", ")
  
  for i in range(len(b)):
   b[i] = int(b[i])
  
  HateScore = sum(b)/len(b)
  return HateScore

In [ ]:
for i in range(5020):
  value = reddit.hatescore.iloc[i]
  HateScore = getscore(value)
  reddit.hatescore.iloc[i] = HateScore

In [ ]:
max(reddit.hatescore)

20.0

Finally lets make the all the hatescores above the value of 0 as value 1.

In [ ]:
HateTweetsIndex = (reddit[reddit["hatescore"] > 0].index)
HateTweetsIndex = HateTweetsIndex.tolist()

for i in HateTweetsIndex:
  reddit.hatescore.iloc[i] = 1

In [ ]:
reddit

,text,hatescore
0,A subsection of retarded Hungarians? Ohh boy....,1
1,"""y'all hear sumn?"" by all means I live in a s...",1
2,wouldn't the defenders or whatever they are a...,0
3,Because the Japanese aren't retarded and know...,1
4,That might be true if we didn't have an examp...,1
...,...,...
5015,Who does she claim as her people? I believe s...,1
5016,Here's what's going to happen: Broward is mag...,0
5017,Oh boy...so here comes a long venting post an...,1
5018,"OP, stop being a faggot and post videos next ...",1


In [ ]:
tweets = pd.read_csv("/content/drive/My Drive/Glue Labs/Data-Comments/Hate Speech English.csv")
tweets

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [ ]:
import re

def PreprocessTweets(tweet):
  n = re.sub(r'~', '', tweet)
  o = re.sub(r'#', '', tweet)
  p = re.sub(r'!+', '', o)
  q = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'LINK', p)
  r = re.sub(r'[*]', '', q)
  s = re.sub(r'[@]\w+', 'USER', r)
  t = re.sub(r'[:|;]', '', s)
  u = re.sub(r'[&]\W+|\d', '', t)
  v = re.sub(r'[&]\W+|\d', '', u)
  w = re.sub(r'[&]\w+', '', v)
  x = re.sub(r'    ', ' ', w)
  y = re.sub(r'   ', ' ', x)
  z = re.sub(r'  ', ' ', y)
  ProcessedTweet = z 
  return ProcessedTweet

In [ ]:
tweets.shape

(24783, 7)

In [ ]:
for i in range(24783):
  text = tweets['tweet'].iloc[i]
  processedtweet = PreprocessTweets(text)
  tweets['tweet'].iloc[i] = processedtweet

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
for i in tweets.columns:
  print(i, ",", max(tweets[i]))

Unnamed: 0 , 25296
count , 9
hate_speech , 7
offensive_language , 9
neither , 9
class , 2
tweet , ~~Ruffled Ntac Eileen Dahlia - Beautiful color combination of pink, orange, yellow white. A Coll LINK


In [ ]:
tweets.tweet[40]

'" momma said no pussy cats inside my doghouse "'

In [ ]:
tweets.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
tweets['hatescore'] = 1.3*tweets['hate_speech'] + tweets['offensive_language'] - tweets['neither'] - 5*tweets['class'] / 5

In [ ]:
OkayTweetsIndex = (tweets[tweets["hatescore"] < 0].index).tolist()
BadTweetsIndex = (tweets[tweets["hatescore"] >= 0].index).tolist()

In [ ]:
for i in OkayTweetsIndex:
  tweets['hatescore'].iloc[i] = 0

for i in BadTweetsIndex:
  tweets['hatescore'].iloc[i] = 1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
tweets.drop(['count', 'hate_speech', 'offensive_language',
             'neither', 'class'], axis = 1, inplace = True)

In [ ]:
tweets.columns = reddit.columns
tweets.index = range(0,24783)
reddit.index = range(24783, 29803)

In [ ]:
commentsdata = pd.concat([tweets, reddit])

In [ ]:
bad = commentsdata[commentsdata['hatescore'] == 1].sample(n = 5000)
good = commentsdata[commentsdata['hatescore'] == 0]
newdata = pd.concat([bad, good])

In [ ]:
pip install fastai --upgrade

     |████████████████████████████████| 194kB 4.8MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
commentsdata.to_csv("/content/drive/My Drive/Glue Labs/Data-Comments/Input/Input#1.csv")

In [ ]:
from fastai.text.all import *
import pandas as pd
import numpy

In [ ]:
classifier = text_classifier_learner(dataloader, AWD_LSTM, metrics = accuracy).to_fp16()

In [ ]:
def probabilty(text):
  Class, tensorClass, probs = classifier.predict(text)
  return print('the class is:', Class, ", with the probability of:", float(max(numpy.array(probs)))*100, '%')